####**Exploring train dataset** 

In [ ]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        n = 0
        for f in files:
            n = n+1
            if n>5:
                print('{}{}'.format(subindent, f),end = "  ...... \n")
                break
            print('{}{}'.format(subindent, f))

img_dir = 'tactile_img/'
list_files("tactile_img/")

In [ ]:
import cv2
import matplotlib.pyplot as plt

PATH_TRAIN =  os.path.join(img_dir, 'TRAIN')

PATH_TEST = os.path.join(img_dir, 'TEST')

normal_sample = cv2.imread(PATH_TRAIN+"/FEMALE/"+os.listdir(PATH_TRAIN+"/FEMALE")[3])
infected_sample = cv2.imread(PATH_TRAIN+"/MALE/"+os.listdir(PATH_TRAIN+"/MALE")[2])

plt.imshow(normal_sample)
plt.title("FEMALE")
plt.show()

plt.imshow(infected_sample)
plt.title("MALE")
plt.show()

The difference can be seen with different visualizations below

In [ ]:
from skimage import io
import matplotlib.pyplot as plt

plt.imshow(normal_sample)
plt.title("FEMALE")
plt.show()

image = io.imread(PATH_TRAIN+"/FEMALE/"+os.listdir(PATH_TRAIN+"/FEMALE")[3])
ax = plt.hist(image.ravel(), bins = 256)
plt.xlabel('Intensity Value')
plt.ylabel('Frequency')
plt.show()

plt.imshow(infected_sample)
plt.title("MALE")
plt.show()

image = io.imread(PATH_TRAIN+"/MALE/"+os.listdir(PATH_TRAIN+"/MALE")[2])
ax = plt.hist(image.ravel(), bins = 256)
plt.xlabel('Intensity Value')
plt.ylabel('Frequency')
plt.show()

####**Initializing the data loader** 

vgg16 pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224.<br />
The images have to be loaded in to a range of [0, 1] and then normalized using:
 #### mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225] <br />
 
 We are applying augmentation like random rotation , horizontal and vertical flips to make sre the model doesn't overfit or learn the wrong features
 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import time
import torchvision
from PIL import ImageFile
import matplotlib.pyplot as plt
import cv2
# ghf
ImageFile.LOAD_TRUNCATED_IMAGES = True # To prevent error during loading broken images

PATH_TRAIN =  os.path.join(img_dir, 'TRAIN')
PATH_TEST = os.path.join(img_dir, 'TEST')

# NOW TEST is a validation set.
EPOCHS = 500
BATCH_SIZE = 32
TOTAL_SIZE = len(os.listdir(PATH_TRAIN + "/FEMALE")) + len(
    os.listdir(PATH_TRAIN + "/MALE")
)
TOTAL_TEST_SIZE = len(os.listdir(PATH_TEST + "/FEMALE")) + len(
    os.listdir(PATH_TEST + "/MALE")
)
STEPS_PER_EPOCH = TOTAL_SIZE // BATCH_SIZE
STEPS_PER_TEST_EPOCH = TOTAL_TEST_SIZE // BATCH_SIZE
IMAGE_H, IMAGE_W = 224, 224

print(TOTAL_SIZE , TOTAL_TEST_SIZE)

In [ ]:
from torchvision.transforms import transforms

transform = transforms.Compose([
    transforms.Resize([224, 224]),                  # Resize input images to a uniform size
    transforms.RandomRotation(degrees=(-10, 10)),   # Randomly rotate the image between -10 and 10 degrees
    transforms.RandomHorizontalFlip(p=0.5),         # Random horizontal flip with a probability of 0.5
    transforms.RandomVerticalFlip(p=0.5),           # Random vertical flip with a probability of 0.5
    transforms.RandomPerspective(distortion_scale=0.6, p=1.0),    # Apply random perspective transformation
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),  # Apply Gaussian blur with randomly chosen parameters
    transforms.ToTensor(),          # Convert PIL Image or numpy.ndarray to tensor and scale to [0, 1]
    transforms.Normalize(           # Normalize to a standard Gaussian distribution to help the model converge faster
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225])   # Mean and std values are computed from a sample of the dataset
])



# Intitalizing the train data loader and applying the transformations

train_dataset = torchvision.datasets.ImageFolder(root=PATH_TRAIN, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, num_workers=1, shuffle=True
)

# Intitalizing the test data loader

test_dataset = torchvision.datasets.ImageFolder(
    root=PATH_TEST, transform=transform
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=BATCH_SIZE, num_workers=1, shuffle=True
)

In [ ]:
from torchvision.models.efficientnet import EfficientNet_V2_S_Weights


model_ft = torchvision.models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.DEFAULT,
                                                progress=True)  # Initializing weight

# fggffg

In [ ]:
#  Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 
#     and may be removed in the future. 
# The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. 
# You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.


num_ftrs = model_ft.classifier[1].in_features # Getting last layer's output features

model_ft.classifier[1] = nn.Linear(num_ftrs, 2) # Modify the last layer to output 2 classes

## **Initializing optimizers and loss function** <br/>
We will also specify the learning rate of the optimiser, here in this case it is set at 0.0001. If our training is bouncing a lot on epochs then we need to decrease the learning rate so that we can reach global minima.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft.to(device)  # Sending model to device
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model_ft.parameters(), lr=0.000001
)  # lr should be kept low so that the pre-trained weights don't change easily

####**TESTING THE MODEL** 

In [ ]:
def get_test():
    test_loss = []
    correct = 0
    incorrect = 0

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    for batch_idx, (data, target) in enumerate(test_loader):
        if batch_idx == STEPS_PER_TEST_EPOCH:
            break

        # Model is used to predict the test data so we are switching off the gradient

        with torch.no_grad():

            data = data.to(device)
            target = target.long().to(device)
            output = model_ft(data)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(output, target)

            # Note that optimizer is not used because the model shouldn't learn the test dataset

            for i in range(BATCH_SIZE):
                a = []
                for j in output[i]:
                    a.append(float(j.detach()))

                pred = a.index(max(a))

                if pred == int(target[i]):
                    correct = correct + 1

                else:
                    incorrect = incorrect + 1

        test_loss.append(float(loss.detach()))
    print("CORRECT: " + str(correct), "INCORRECT: " + str(incorrect),"TEST ACCURACY: "+str(correct/(correct+incorrect)))
    return (
            correct/(incorrect+correct),
            sum(test_loss)/len(test_loss),
    )





In [ ]:
acc_ , loss_ = get_test()
print("ACCURACY AND LOSS BEFORE TUNING")
print("ACCURACY : "+str(acc_),"LOSS : "+str(loss_))

####**TUNING THE MODEL (TRAINING)** 

In [ ]:
avg_test_loss_history = []
avg_test_accuracy_history = []
avg_train_loss_history = []
avg_train_accuracy_history = []

loss_history = []
accuracy_history = []

new_best = 0

for i in range(EPOCHS):

    start = time.time()
    print(
        "-----------------------EPOCH "
        + str(i)
        + " -----------------------------------"
    )
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx == STEPS_PER_EPOCH:
            break
        optimizer.zero_grad()  # Resetting gradients after each optimizations
        # Sending input , target to device
        data = data.to(device) 
        target = target.to(device)
        output = model_ft(data)
        loss = criterion(output, target.reshape((BATCH_SIZE,)).long())
        loss_history.append(loss.detach())
        # The loss variable has gradient attached to it so we are removing it so that it can be used to plot graphs
        loss.backward()
        optimizer.step()  # Optimizing the model

        # Checking train accuracy

        correct = 0
        incorrect = 0
        for p in range(BATCH_SIZE):
            a = []
            for j in output[p]:
                a.append(float(j.detach()))

            pred = a.index(max(a))

            if pred == int(target[p]):
                correct = correct + 1

            else:

                incorrect = incorrect + 1

        print(
            "\r EPOCH "
            + str(i)
            + " MINIBATCH: "
            + str(batch_idx)
            + "/"
            + str(STEPS_PER_EPOCH)
            + " LOSS: "
            + str(loss_history[-1]),
            end = ""
            
        )
        
        accuracy_history.append(correct/(correct+incorrect))

    end = time.time()
    print(
        " \n EPOCH "
        + str(i)
        + " LOSS "
        + str(sum(loss_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)
        + " ETA: "
        + str(end - start)
        + " \n MAX LOSS: "
        + str(max(loss_history[-STEPS_PER_EPOCH:]))
        + " MIN LOSS: "
        + str(min(loss_history[-STEPS_PER_EPOCH:]))
        + " TRAIN ACCURACY: "
        + str(sum(accuracy_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)
    )
    
    avg_train_loss_history.append(sum(loss_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)
    avg_train_accuracy_history.append(sum(accuracy_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)
    
    test_acc , test_loss  = get_test()
    
    avg_test_accuracy_history.append(test_acc)
    avg_train_loss_history.append(test_loss)
    
    if test_acc>new_best: 
        new_best = test_acc
        torch.save(model_ft.state_dict(), "_lr0.0001_efficient.pth") # Saving our best model



#### Plotting the accuracy produced by model during training

In [ ]:
plt.plot(avg_train_accuracy_history , label = "Train")
plt.plot(avg_test_accuracy_history , label = "Test")
plt.title('ACCURACY PER EPOCH')
plt.xlabel("EPOCHS")
plt.ylabel("ACCURACY")
plt.legend(loc="upper left")


In [ ]:
print('updated.')